# Flank Quickstart on Google Colab

Create an example API, serve it over the broader internet, and then create a webpage for interacting with it using Flank.

# **1A. Get an API running in this browser 🏃**

---

First, we're going to create a little API for creating and retrieving sales data. It's a simple example, but it's representative of what a company's internal systems might look like.

In [ ]:
%pip install -q fastapi uvicorn pyngrok # This could take a minute

In [ ]:
import uvicorn
import getpass
import json
import requests
from fastapi import FastAPI, Query
from typing import Optional
import datetime

server_instance = None

app = FastAPI()

# Dummy sales data
sales_data = [
    {"id": 1, "date": "2023-01-01", "amount": 100.0, "customer_name": "ABC Corp"},
    {"id": 2, "date": "2023-01-02", "amount": 150.0, "customer_name": "XYZ Inc"},
    {
        "id": 3,
        "date": "2023-01-03",
        "amount": 200.0,
        "customer_name": "LMN Enterprises",
    },
    {"id": 4, "date": "2023-01-04", "amount": 120.0, "customer_name": "PQR Limited"},
    {
        "id": 5,
        "date": "2023-01-05",
        "amount": 180.0,
        "customer_name": "EFG Corporation",
    },
]


# Get a specific sale
@app.get("/api/sales/{id}")
async def get_sale(id: int) -> dict:
    for sale in sales_data:
        if sale["id"] == id:
            return sale
    return None


# List of sales
@app.get("/api/sales")
async def get_sales(
    start_date: Optional[str] = None, end_date: Optional[str] = None
) -> list:
    filtered_sales = [
        sale
        for sale in sales_data
        if (start_date is None or sale["date"] >= start_date)
        and (end_date is None or sale["date"] <= end_date)
    ]
    return filtered_sales


# Create a new sale
@app.post("/api/sales")
async def add_sale(date: str, amount: float, customer_name: str) -> dict:
    sale = {
        "id": max([d["id"] for d in sales_data]) + 1,
        "date": date,
        "amount": amount,
        "customer_name": customer_name,
    }
    sales_data.append(sale)
    return sale


# Run this API locally on port 8050
def run_server():
    global server_instance
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8050)
    server_instance = uvicorn.Server(config)
    server_instance.run()


def stop_server():
    if server_instance is not None:
        server_instance.should_exit = True
        server_instance.force_exit = True
        print("Server is shutting down...")
    else:
        print("Server is not running.")

In [ ]:
import threading
local_thread = threading.Thread(target=run_server)
local_thread.start()

# **1B. Check out your (in-browser) API ✅**

---

Run the following cell and go to the URL that it spits out *in this browser*. It won't work in a browser where you aren't logged into Google, or on someone else's computer.

In [ ]:
from google.colab.output import eval_js

# See your endpoint running locally
url = eval_js("google.colab.kernel.proxyPort(8050)")
print(f"View one sale here: {url}api/sales/1")
print(f"View list of sales here: {url}api/sales?start_date=2023-01-02")

# **🔍 Closer look at what just happened 🔍**

---

You just created a simple API so that we have something to work with. To clarify, the point of this quickstart is **not** to simply create an API -- it's to show how quickly Flank can integrate with existing APIs (that's coming soon).


### Explanation of code:
- An **API** is a collection of functions that can be called over the internet using HTTP, just like how you type a website URL into a browser. Typically, each URL path ("/sales/{id}" versus "/sales") corresponds to a different function.
- **FastAPI** is an API framework. It provides a "language" within Python for writing APIs in a simple, straightfowrad way.
- **uvicorn** is a server that listens for API requests. It forwards requests to whatever application it's running (the FastAPI `app` in this case)
- `@app.get("/api/sales/{id}")` is FastAPI code that says, in effect, *"Create an endpoint around the following Python function (in this case, `get_sale`) and expose it on the URL path `/sales/{id}`"*.
- `threading` is a built-in Python module that creates new threads within a process. We run the API within a new thread because, if we don't, the API will run continuously on the main thread, which will block any Jupyter Notebook cells further down the page.
- You might have noticed that there are two Python functions mapped to the same URL path -- "/api/sales". How does the API determine which Python function to call when it gets a request on that URL?
  - Well, there are actually several types of HTTP requests. When you punch a URL into the browser that is a "GET" request.
  - FastAPI configures a "GET" request like this: `@app.get()`.
  - "POST" is another type of HTTP request, which explains how the server distinguises between requests on "/api/sales" -- `@app.post()`. 

# **2A. Expose that API over the public internet 🌎**

---

We're going to use a service called ngrok to take our local API and make it available over the public internet.

### Todos:
1. In a new tab, [log in or create a free ngrok account](https://dashboard.ngrok.com/get-started/your-authtoken). 
2. Copy the Authtoken at the top of screen here: [authtoken page](https://dashboard.ngrok.com/get-started/your-authtoken)
3. Run the code below and paste it in when prompted

In [ ]:
from pyngrok import ngrok, conf
print("Paste in your ngrok authtoken that you copied in the step above, then hit enter:")
conf.get_default().auth_token = getpass.getpass()

In [ ]:
ngrok_tunnel = ngrok.connect(8050)

# **2B. Check out your public API! ✅**

---

Run the code below. Now, your API should be visible from *any* computer on the internet.

In [ ]:
ngrok_url = ngrok_tunnel.public_url
print('Endpoint exposed publicly at:', ngrok_url + "/api/sales/1")

# **🔍 Closer look at what just happened 🔍**

---

When you run a web server on your computer (or a Google Colab server), the service isn't necessarily visible to the broader internet, even though **you** can see the broader internet from your computer. `ngrok` is a developer tool that lets you expose local servers to the broader internet.

### Explanation of code:
- Even though they have a free tier, `ngrok` doesn't allow any anonymous person to use their service. To use `ngrok` you need to establish a connection that says "I am this person". That was the purpose of copying the Authtoken from the ngrok website and pasting it into this notebook.
- `ngrok` creates a "tunnel" from your local computer (or the Colab server) to the public internet
- You may be running multiple servers on your computer, and you may not wish to expose ALL of them to the broader internet. Thus the concept of "ports". Ports give you the ability to create lots of network connections and configure each of them separately.
- In this case, we ran the API server on port 8050, thus `ngrok.connect(8050)` creates a tunnel to port 8050 on your server.

# **3A. Get a Flank API token 🪪**

---

### Todos

1. Go to [this page in Flank](https://www.flank.cloud/copy-token?goStraightToAuth0) and an `auth_token` and `org_id` will get copied to your clipboard
2. Paste below and run the cell

In [ ]:
# Paste below to replace empty values
auth_token = ""
org_id = ""

# **3B. Create Flank Apps 🪄**

---

Now we're going to create "apps" in Flank for each of the endpoints. It should only take about 10 seconds.

In [ ]:
flank_api_url = f"https://api.flank.cloud/v1/{org_id}"
docs_endpoint = f"{ngrok_url}/openapi.json"
headers = {"Authorization": f"Bearer {auth_token}"}

resources_payload = {
    "name": "Sales API",
    "description": "",
    "creator_org_id": org_id,
    "resource_type": "api",
    "is_public": True,
    "api": {"api_spec_url": docs_endpoint, "image_url": "", "api_auth_schema": ""},
    "db": None,
}

add_resources_resp = requests.post(
    url=f"{flank_api_url}/resources",
    headers=headers,
    data=json.dumps(resources_payload),
)
add_resources_resp = add_resources_resp.json()
resource_id = add_resources_resp["resource_id"]

creds_payload = {
    "org_id": org_id,
    "cred_type": "api",
    "name": "Sales API",
    "syncs": True,
    "runs": True,
    "stores": False,
    "creds_db": None,
    "creds_aws": None,
    "creds_az": None,
    "creds_api": {
        "custom_headers": "",
        "auth_type": None,
        "api_spec_url": docs_endpoint,
        "client_key": "",
        "client_secret": "",
        "bearer_token": "",
        "api_key": None,
        "username": None,
        "password": None,
    },
    "resource_id": resource_id,
}
creds_endpoint_resp = requests.post(
    url=f"{flank_api_url}/creds", headers=headers, data=json.dumps(creds_payload)
)
creds_endpoint_resp = creds_endpoint_resp.json()

creds_id = creds_endpoint_resp["creds_id"]
sync_resource_resp = requests.get(
    url=f"{flank_api_url}/creds/{creds_id}/diff", headers=headers
)
sync_resource_resp = sync_resource_resp.json()

kit_ids = []

for endpoint in sync_resource_resp:
    sync_endpoint_payload = {"cloud_id": endpoint["cloud_id"], "creds_id": creds_id}
    sync_endpoint_resp = requests.post(
        url=f"{flank_api_url}/syncs/cloud_id",
        headers=headers,
        data=json.dumps(sync_endpoint_payload),
    )
    sync_endpoint_resp = sync_endpoint_resp.json()

    kit_id = sync_endpoint_resp["stage2"]["kit_id"]
    kit_ids.append(kit_id)

# **3C. Check out your Flank app ✅**

---

Now you have shareable, runnable apps in Flank, that connect to the API you created!

In [ ]:
for kit_id in kit_ids:
    print(
        f"Check out your Flank UI at https://www.flank.cloud/{org_id}/builder?id={kit_id}"
    )

# **🔍 Closer look at what just happened 🔍**

---

First you created an API. Then you exposed that API to the broader internet using ngrok. Then, you pointed Flank at that API and Flank automatically created web pages for each endpoint.

Under the hood, Flank used an OpenAPI spec, which is created by FastAPI. 

# **Where to go from here 🚏**

---

- **Tweak the code and see the changes take effect.** You can run `stop_server` (see below) and then re-run the code in section 1A.
- **Host an API permanently.** This API will die when you shut down this notebook.
- **Stitch endpoints together to create more functionality.** For example, a dropdown that ensures that someone doens't fat-finger the `sales_id`. Or a table of sales, where someone can take actions on each row.
- **Share this with the world.** In Flank, Share > Store / Public
- **Share this internally with a teammate.** In Flank, Share > This Org

In [ ]:
# If you need to shut down your local server (optional)
stop_server()